# LDA - GBT Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from load_dataset import load_dataset

## Constants

In [2]:
RANDOM_SEED = 0
DATASET_PATH = "dataset/hate_speech_filipino_dataset.pkl"

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
X, Y = load_dataset(DATASET_PATH)

In [5]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [6]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

## Perform Latent Dirichlet Allocation on the training set

In [7]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
X_val_topics = lda.transform(X_val_counts)
X_test_topics = lda.transform(X_test_counts)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


## Train the Gradient-boosting Tree

In [9]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
        max_depth = trial.suggest_int("max_depth", 10, 100)
        gbt = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=RANDOM_SEED)
        gbt.fit(X_train, Y_train)
        Y_pred = gbt.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [10]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_GBT_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 00:15:41,522] A new study created in memory with name: LDA_GBT_Pipeline
[I 2024-04-30 00:18:04,749] Trial 7 finished with value: 0.6983078827899298 and parameters: {'n_estimators': 388, 'learning_rate': 0.0011647439451420093, 'max_depth': 11}. Best is trial 7 with value: 0.6983078827899298.
[I 2024-04-30 00:18:23,169] Trial 2 finished with value: 0.8378043747420553 and parameters: {'n_estimators': 115, 'learning_rate': 0.007635154042794744, 'max_depth': 70}. Best is trial 2 with value: 0.8378043747420553.
[I 2024-04-30 00:20:31,761] Trial 6 finished with value: 0.7870408584399504 and parameters: {'n_estimators': 798, 'learning_rate': 0.005897766936296455, 'max_depth': 12}. Best is trial 2 with value: 0.8378043747420553.
[I 2024-04-30 00:22:13,328] Trial 13 finished with value: 0.8382170862567065 and parameters: {'n_estimators': 152, 'learning_rate': 0.003592720326893685, 'max_depth': 37}. Best is trial 13 with value: 0.8382170862567065.
[I 2024-04-30 00:23:02,396] Trial 1

Best Accuracy: 0.8386297977713578
Best hyperparameters: {'n_estimators': 310, 'learning_rate': 0.00919283524285072, 'max_depth': 94}


In [11]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]
best_max_depth = study.best_trial.params["max_depth"]

gbt = GradientBoostingClassifier(n_estimators=best_n_estimators, 
                                 learning_rate=best_learning_rate,
                                 max_depth=best_max_depth,
                                 random_state=RANDOM_SEED)
gbt.fit(X_train_topics, Y_train)
Y_pred = gbt.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.8638613861386139


In [6]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# 
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
# X_train_vector = tfidf_vectorizer.fit_transform(X_train)

In [7]:
from sklearn.decomposition import NMF

N_TOPICS = 20
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
X_val_topics = nmf.transform(X_val_counts)
X_test_topics = nmf.transform(X_test_counts)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 10 topics
Done performing Non-negative Matrix Factorization for 10 topics


In [9]:
gbt = GradientBoostingClassifier(n_estimators=310, 
                                 learning_rate=0.00919283524285072,
                                 max_depth=94,
                                 random_state=RANDOM_SEED)
gbt.fit(X_train_topics, Y_train)
Y_pred = gbt.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best NMF-GBT Model Accuracy: {score}")

Best LDA-GBT Model Accuracy: 0.8634488448844885
